# 4c. Daten beschaffen

In diesem Notebook setzen wir die Datenbeschaffung fort, indem wir die Volltexte der Märchen strukturieren.

## Inhalte
- Wir laden die zuvor erzeugte TSV-Datei
- Wir bereiten mehrere leeren Spalten vor
- Wir laden jedes HTML-Dokument in ein BeautifulSoup-Objekt
- Wir durchsuchen das BeautifulSoup-Objekt nach bibliographischen Metadaten und dem Volltext des Märchens
- Wir fügen diese Informationen in die Tabelle ein
- Wir speichern die strukturierte Tabelle für den nächsten Schritt


In [1]:
import pathlib
import re
import time

import pandas as pd
import requests
from bs4 import BeautifulSoup

DATA_DIR = pathlib.Path().cwd().parent.joinpath("data")
RAW_DATA_DIR = DATA_DIR.joinpath("raw")
PROC_DATA_DIR = DATA_DIR.joinpath("processed")

In [2]:
# read data from TSV file on disk into a pandas.DataFrame
filename = RAW_DATA_DIR.joinpath("wikisource_fairytale_html.tsv")
fairytales = pd.read_csv(filename, sep="\t")

# initialize empty columns
fairytales["title"] = None
fairytales["creator"] = None
fairytales["date"] = None
fairytales["place"] = None
fairytales["edition"] = None
fairytales["publisher"] = None
fairytales["source"] = None
fairytales["text"] = None

fairytales.head()

,url,html,title,creator,date,place,edition,publisher,source,text
0,https://de.wikisource.org/wiki/%C3%84skenbride...,"<div class=""mw-parser-output""><div class=""nopr...",None,None,None,None,None,None,None,None
1,https://de.wikisource.org/wiki/Ahmed_und_Paribanu,"<div class=""mw-parser-output""><div class=""nopr...",None,None,None,None,None,None,None,None
2,https://de.wikisource.org/wiki/Albanesische_M%...,"<div class=""mw-parser-output""><div class=""nopr...",None,None,None,None,None,None,None,None
3,https://de.wikisource.org/wiki/Allerlei-Rauh_(...,"<div class=""mw-parser-output""><div class=""nopr...",None,None,None,None,None,None,None,None
4,https://de.wikisource.org/wiki/Allerlei-Rauh_(...,"<div class=""mw-parser-output""><div class=""nopr...",None,None,None,None,None,None,None,None


In [3]:
def get_metadata(table):
    """Extract bibliographic metadata from HTML table."""
    metadata = {
        "creator": None,
        "title": None,
        "date": None,
        "place": None,
        "edition": None,
        "publisher": None,
        "source": None,
    }

    for table_row in table.find_all("tr"):
        table_columns = table_row.find_all("td")
        if table_columns and len(table_columns) == 2:
            metada_element_name = table_columns[0].text.strip()
            metada_element_value = table_columns[1].text.strip()
            if metada_element_name == "Titel:":
                metadata["title"] = metada_element_value
            if metada_element_name == "Autor:":
                metadata["creator"] = metada_element_value
            if metada_element_name == "Erscheinungsdatum:":
                # Some dates are surrounded by brackets: [1820] => 1820
                date = metada_element_value
                date = re.search("\d+", date)
                if date:
                    date = int(date.group())
                else:
                    date = None
                metadata["date"] = date
            if metada_element_name == "Erscheinungsort:":
                metadata["place"] = metada_element_value
            if metada_element_name == "Auflage:":
                metadata["edition"] = metada_element_value
            if metada_element_name == "Verlag:":
                publisher = metada_element_value
                metadata["publisher"] = metada_element_value
            if metada_element_name == "aus:":
                metadata["source"] = metada_element_value
    return metadata

In [4]:
def get_content_text(content):
    """Extract fulltext from HTML"""

    # remove unwanted sections from text
    spoken_text = content.find(id="GesprochenerText")
    if spoken_text:
        spoken_text.decompose()

    # remove reference list of other editions
    other_editions = content.find(
        string=re.compile("Andere Ausgaben unter diesem Titel siehe unter")
    )
    if other_editions:
        other_editions.parent.decompose()

    # remove links, e.g. [408], [409]
    for td in content.find_all("span", class_="PageNumber"):
        td.decompose()

    # remove bold text in centered text section
    # these sections repeat the fairytale's title in its fulltext.
    for centered_text in content.find_all("div", {"style": re.compile("text-align:center")}):
        for bold_text in centered_text.find_all("b"):
            bold_text.clear()

    # remove appendices
    appendix_regex = re.compile("Anhang|Anmerkung")
    first_appendix = soup.find("h2", string=appendix_regex)
    if not first_appendix:
        first_appendix = soup.find("span", string=appendix_regex)

        if first_appendix and first_appendix.parent.name == "h2":
            first_appendix = first_appendix.parent

    if first_appendix:
        for sibling in first_appendix.next_siblings:
            try:
                sibling.clear()
            except AttributeError:
                pass
        first_appendix.decompose()

    text = content.text.strip()
    text = re.sub("\n ", " ", text)

    # Considering: replace german quotation marks
    # text = re.sub("„|“", '"', text)
    # text = text.strip()
    return text

In [5]:
for row_id, row in fairytales.iterrows():
    soup = BeautifulSoup(row["html"], "html.parser")
    content = soup.find("div", class_="mw-parser-output")

    metadata_section = content.find(id="textdaten")
    if metadata_section:
        metadata_table = metadata_section.find("table").find("table")
        metadata = get_metadata(metadata_table)
        metadata_section.decompose()

        # add data to dataframe
        fairytales.at[row_id, "title"] = metadata["title"]
        fairytales.at[row_id, "creator"] = metadata["creator"]
        fairytales.at[row_id, "date"] = metadata["date"]
        fairytales.at[row_id, "place"] = metadata["place"]
        fairytales.at[row_id, "edition"] = metadata["edition"]
        fairytales.at[row_id, "publisher"] = metadata["publisher"]
        fairytales.at[row_id, "source"] = metadata["source"]

    text = get_content_text(content)
    fairytales.at[row_id, "text"] = text

In [6]:
fairytales.head()

,url,html,title,creator,date,place,edition,publisher,source,text
0,https://de.wikisource.org/wiki/%C3%84skenbride...,"<div class=""mw-parser-output""><div class=""nopr...",Äskenbridel Saúnsidel,Heinrich Georg Ehrentraut,1854,Oldenburg,,Vorlage:none,"Mittheilungen aus der Sprache der Wangeroger, ...",Auf der Watteninsel Wangerooge wurde bis um 19...
1,https://de.wikisource.org/wiki/Ahmed_und_Paribanu,"<div class=""mw-parser-output""><div class=""nopr...",Ahmed und Paribanu,Johann Andreas Christian Löhr,1820,Leipzig,1. Auflage,Gerhard Fleischer d. Jüng.,Das Buch der Maehrchen für Kindheit und Jugend...,Ein mächtiger König von Indien hatte drei Söhn...
2,https://de.wikisource.org/wiki/Albanesische_M%...,"<div class=""mw-parser-output""><div class=""nopr...",Albanesische Märchen,Wilhelm Grimm,1853,Göttingen,,Dieterische Buchhandlung,Zeitschrift für deutsche Mythologie und Sitten...,Allmälig werden uns auch die überlieferungen s...
3,https://de.wikisource.org/wiki/Allerlei-Rauh_(...,"<div class=""mw-parser-output""><div class=""nopr...",Allerlei-Rauh,Brüder Grimm,1812,Berlin,1. Auflage,Realschulbuchhandlung,"Kinder- und Haus-Märchen Band 1, Große Ausgabe...","Es war einmal ein König, der hatte eine Frau, ..."
4,https://de.wikisource.org/wiki/Allerlei-Rauh_(...,"<div class=""mw-parser-output""><div class=""nopr...",Allerlei-Rauh,Brüder Grimm,1819,Berlin,2. Auflage,G. Reimer,"Kinder- und Haus-Märchen Band 1, Große Ausgabe...","Es war einmal ein König, dessen Frau hatte Haa..."


In [7]:
# sort the table's contents by multiple columns
sort_by = ["creator", "date", "title"]
fairytales.sort_values(by=sort_by, inplace=True)
fairytales.reset_index(inplace=True, drop=True)
# drop the HTML column
fairytales.drop("html", axis="columns", inplace=True)
fairytales.head()

,url,title,creator,date,place,edition,publisher,source,text
0,https://de.wikisource.org/wiki/Des_Perlenfisch...,Des Perlenfischers Töchterlein,,1845,München,,Braun & Schneider,"Fliegende Blätter, Band 1, Nr. 12, S. 89–92 un...","In Bayern ist ein Ländlein, heißt die Steinpfa..."
1,"https://de.wikisource.org/wiki/Bittet,_so_wird...","Bittet, so wird Euch gegeben!",,1930,Danzig,,Fuchs & Cie.,"Aus dem Märchenschatz der Kaschubei, S. 43–46","Es war einmal ein hochbetagter Mann, der war s..."
2,https://de.wikisource.org/wiki/Der_Soldat_und_...,Der Soldat und der Teufel,,1930,Danzig,,Fuchs & Cie.,"Aus dem Märchenschatz der Kaschubei, S. 25–26",Einmal stand ein Soldat auf Posten. Es war im ...
3,https://de.wikisource.org/wiki/Der_gute_Zeuge,Der gute Zeuge,,1930,Danzig,,Fuchs & Cie.,"Aus dem Märchenschatz der Kaschubei, S. 37–39",Ein junger Mensch ging auf Wanderschaft. Als e...
4,https://de.wikisource.org/wiki/Der_gute_und_de...,Der gute und der böse Bruder,,1930,Danzig,,Fuchs & Cie.,"Aus dem Märchenschatz der Kaschubei, S. 31–35","Es waren einmal zwei Brüder, die wollten in di..."


In [8]:
# store the urls and metadata in a TSV file inside the data/ directory
filename = PROC_DATA_DIR.joinpath("wikisource_fairytale_fulltext.tsv")
fairytales.to_csv(filename, sep="\t", index=False)

In [9]:
import session_info

session_info.show()